# -------------------------隐马尔可夫模型解决北京市车辆交通预测问题------------------------

## 声明

* 由于使用经纬度表示和计算并不方便，本分析中一律采用m为单位，如需转换为经纬度，请使用我提供的转换函数

# 预备

思路：<br>
<br>
隐藏的状态：车的速度、方向等因素（但这是连续值，是否需要思考如何转为‘堵车’‘畅通’等离散值）<br>
可见的观测：车的位置、位移（距上次测量的位移）<br>
<br>
A矩阵：状态转移矩阵，指车的状态切换矩阵，如从堵车状态转移为畅通状态<br>
B矩阵：发射（观测）矩阵，指状态显现出的形式，这里采用高斯分布<br>
    <br>
大体算法：<br>
1、求出每个点的速度、方向、速度较上一点的改变量、方向较上一点的改变量、原地停留时间(v,theta,u,diata,stops_time)<br>
2、用维特比算法对(x,y,v,theta,u,diata)进行隐马尔可夫模型训练<br>
3、通过后向算法进行预测<br>

## 数据预处理

预处理中出现了问题，tdf和ctdf的stops_time有误，需修改

In [83]:
%run data_pretreatment.py
%reset -f
print 'all variables have been removed'

loading train_set...
transforming lon and lat to x and y...
computing v and u...
computing stops_time and restart
fixing the error number...
df has been output
cdf has been ouput
loading test_set...
transforming lon and lat to x and y...
computing v and u...
computing stops_time and restart
fixing the error number...
tdf has been output
ctdf has been ouput
data_pretreatment has been done
all variables have been removed


## 预备库环境

In [86]:
%pylab
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series,DataFrame
import seaborn as sns
from pprint import pprint as pp
import scipy as sp
import matplotlib as mpl
from hmmlearn import hmm
import math
from sklearn.externals import joblib

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [136]:
import warnings
warnings.filterwarnings('ignore')

In [145]:
%run funcs.py

## 回顾上次实验的结果

对于用训练集中前500个数据作为训练集，训练集中后1500个数据作为测试集的模型

取上一点的state直接当做下一点的state的结果是：16.689852160660948<br>
取proba最大值的state作为state的结果是：16.463392959141991<br>
取v均值作为v的结果是：14.644368553942261<br>
<br>

使用所有源训练集的数据作为训练集,使用所有源训练集的数据作为测试集的结果：<br>
分成8个状态的结果<br>
取proba最大值的state作为state的结果是：14.2193538343857  (疑有误，可能为取v期望作为v的结果)<br>
分成16个状态的结果<br>
取v期望的结果是：16.418680700780275<br>


model      | model1      | model2 |model3|model4|model5
------------ | ------------- | ------------ | ------------ 
discribe    |  取上一点的state直接当做下一点的state | 取proba最大值的state作为state|取v的期望|取proba最大值的state作为state的结果|取v的期望
component   | 8|8|8 |8|16
item       | ||||200
n_item     | 100|100|100|100|200
trainset    | df[:500]  |  df[:500] | df[:500] |df|df
testset    | df[500:] | df[500:] |df[500:] |tdf|tdf
err_distance | 16.689852160660948  | 16.463392959141991 | 14.644368553942261|14.2193538343857|16.418680700780275



分得更细，反倒效果变差了<br>
<br>
Interesting！<br>
由于本例中存在很多v为0的点，当采用期望后这些v都被预测为4.5<br>
所以我认为去v期望作为v的结果会很差，但是事实却相反<br>
这可能说明：<br>
* 第二种方法的极大误差值严重影响了错误率
* 对于观测值为连续值的问题，用连续的期望代替离散值有天然的优越性
* 将第二种方法和第三种方法结合可能会有更好的效果
* 本方法还有很大的提升空间

现在demo跑出来了，下一步要优化<br>
优化想法：<br>
* 加入其它特征进行训练
* 将训练集训练出的模型和测试集中将要预测的轨迹的点（除去要预测的点）训练出的小模型融合，这样的效果可能会更贴切该车的行车规律，结果可能更好
* 调参

## 从middata中加载数据

In [89]:
df = pd.read_csv('../middata/df3.csv',index_col=0)   # index_col=0 可以去掉Unnamed0
cdf = pd.read_csv('../middata/cdf1.csv',index_col=0)
tdf = pd.read_csv('../middata/tdf4.csv',index_col=0)
ctdf = pd.read_csv('../middata/ctdf2.csv',index_col=0)

In [122]:
# 生成用于测试代码的测试数据
# 测试训练集
fff = cdf[cdf['id']<20]
# 测试测试集
ttt = ctdf[ctdf['id']<20]
# 并储存
%store fff
%store ttt

Stored 'fff' (DataFrame)
Stored 'ttt' (DataFrame)


## 从models中加载模型

在刚开始的时候没有注意保存的模型命名的问题，尤其是模型对应的traincol。

以下是整理后的各模型参数：

model                                 |trainset| traincol    |n_components | n_iter          |create_time
-----------------------------------------------------|-------|--|----------|----------|------------------------------------------
model_2000_car_300_iter_v_u_diata_12m2d21h31min.pkl  | cdf     |v u diata    |16        | 300           |12m2d21h31min
model_test_v_u_diata_stops_12m2d19h48min.pkl       |cdf[:100*1600]|v u diata |12        | 50            |12m2d19h48min
model_2000_car_200_iter_12m1d7h47min.pkl         | cdf     |v         |16        | 200           |12m1d7h47min
model_2000_car_100_iter_v.pkl                   | cdf     |v         |8         |100           |-

In [138]:
model_2000_car_100_iter_v = joblib.load('../models/model_2000_car_100_iter_v.pkl')
model_2000_car_200_iter_12m1d7h47min = joblib.load('../models/model_2000_car_200_iter_12m1d7h47min.pkl')
model_test_v_u_diata_stops_12m2d19h48min = joblib.load('../models/model_test_v_u_diata_stops_12m2d19h48min.pkl')
model_2000_car_300_iter_v_u_diata_12m2d21h31min = joblib.load('../models/model_2000_car_300_iter_v_u_diata_12m2d21h31min.pkl')

表中1,2模型类型相同，3,4模型类型相同<br>
实验表明，其中1,3模型较2,4模型更好<br>
故这里只选取1,3模型进行展示

## 测试

### 多特征训练

为了加入其它特征进入训练，我同时考虑了速度、加速度、方向改变量、原地停顿时间四个因素,但是结果很不理想，误差很大<br>
没有演示实验，因为用于测试的模型比较粗糙，没有保留<br>
精度比较高的模型还正在训练(已经训练一两天了，不知道还要训练多久)<br>
于是我同时考虑了速度、加速度、方向改变量三个因素,但是误差还是很大<br>
如下测试所见

In [146]:
prediction(model_2000_car_300_iter_v_u_diata_12m2d21h31min,cdf,ctdf,traincol_name=['v','u','diata'])

---------*观察报告*---------
每种状态对应的轨迹数
状态 0 有 16345775 条轨迹
状态 1 有 282462 条轨迹
状态 2 有 374982 条轨迹
状态 3 有 116518 条轨迹
状态 4 有 156263 条轨迹
状态 5 有 255575 条轨迹
状态 6 有 42723 条轨迹
状态 7 有 547820 条轨迹
状态 8 有 782806 条轨迹
状态 9 有 255734 条轨迹
状态 10 有 362202 条轨迹
状态 11 有 534367 条轨迹
状态 12 有 170095 条轨迹
状态 13 有 237555 条轨迹
状态 14 有 387425 条轨迹
状态 15 有 147698 条轨迹
每种状态对应的观测
每条轨迹长度: 2100
轨迹数: 10000
观测状态矩阵
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  5.42297393e+01   4.98010739e+01  -2.43646686e+00   0.00000000e+00]
 [  6.23644912e+01   1.78507821e+01   9.24222568e+01   0.00000000e+00]
 [  2.75682832e+01   3.24730373e+02   4.19715592e+00   0.00000000e+00]
 [  3.43089099e+01   1.53915221e+01   9.42156487e-12   0.00000000e+00]
 [  9.77850836e+01   8.42988064e+01   9.70866390e-01   0.00000000e+00]
 [  8.44573181e+02   7.60760568e+02  -2.40655108e+00   0.00000000e+00]
 [  2.69784094e+01   1.92395809e+01   9.01738316e+01   0.00000000e+00]
 [  0.00000000e+00   4.42073448e+01  -1.25877365e+01   0.000

加速度和其角度diata必须同时考虑，不能拆开<br>
所以我只好只考虑速度v

In [149]:
prediction(model_2000_car_200_iter_12m1d7h47min,cdf,ctdf,traincol_name=['v'])

---------*观察报告*---------
每种状态对应的轨迹数
状态 0 有 17157462 条轨迹
状态 1 有 480867 条轨迹
状态 2 有 266683 条轨迹
状态 3 有 616035 条轨迹
状态 4 有 52 条轨迹
状态 5 有 247124 条轨迹
状态 6 有 16251 条轨迹
状态 7 有 0 条轨迹
状态 8 有 218691 条轨迹
状态 9 有 571654 条轨迹
状态 10 有 103610 条轨迹
状态 11 有 94408 条轨迹
状态 12 有 212014 条轨迹
状态 13 有 264689 条轨迹
状态 14 有 750319 条轨迹
状态 15 有 141 条轨迹
每种状态对应的观测
每条轨迹长度: 2100
轨迹数: 10000
观测状态矩阵
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  6.10505636e+01   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  3.09428371e+02   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  4.38698485e+01   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  9.17854605e+03   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.56911230e+02   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.19635438e+03   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  3.00105223e+04   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  8.94227970e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [

效果在预期范围内

### 模型融合

由于使用模型融合的方法计算量较大，所以这里仅取部分数据进行检测<br>
我正在跑检测所有测试集的程序，可能明天会跑出全部测试集的训练效果

In [150]:
self_prediction(model_2000_car_200_iter_12m1d7h47min,cdf,cdf[cdf['id']<50],traincol_name=['v'],weight_predict_base_all=0.5)

---------*观察报告*---------
每种状态对应的轨迹数
状态 0 有 60184 条轨迹
状态 1 有 2374 条轨迹
状态 2 有 1126 条轨迹
状态 3 有 3457 条轨迹
状态 4 有 0 条轨迹
状态 5 有 1179 条轨迹
状态 6 有 59 条轨迹
状态 7 有 0 条轨迹
状态 8 有 1059 条轨迹
状态 9 有 3000 条轨迹
状态 10 有 480 条轨迹
状态 11 有 389 条轨迹
状态 12 有 1308 条轨迹
状态 13 有 1282 条轨迹
状态 14 有 4102 条轨迹
状态 15 有 1 条轨迹
每种状态对应的观测
每条轨迹长度: 1600
轨迹数: 50
观测状态矩阵
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  6.10505636e+01   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  3.09428371e+02   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  4.38698485e+01   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  9.17854605e+03   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.56911230e+02   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  1.19635438e+03   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  3.00105223e+04   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  8.94227970e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]
 [  6.78816256e+01   0.00000000e+00  

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

很不幸，报错了，因为main.ipynb文件是用来展示用的，我的探索、尝试部分是分散在其他几个文件之间的，<br>
这可能导致main.ipynb文件的变量环境和其他文件的环境稍有区别,或其他问题<br>
若要看到这里的数据，请移步fit.ipynb文件的第302块代码<br>
这部分的数据显示，融合后的结果和融合前的结果相近，但错误率稍高

### 调参

这里有好几个参数可调，但是调参太花费时间了，我这里只展示调高weight_predict_base_all后的效果

In [ ]:
self_prediction(model_2000_car_200_iter_12m1d7h47min,cdf,cdf[cdf['id']<50],traincol_name=['v'],weight_predict_base_all=0.85)

既然上面的代码会报错，这里的代码就不尝试了<br>
结果是稍好于weight_predict_base_all=0.5的测试结果，但还不如融合前的结果<br>
这说明融合的效果确实不如不融合

# 总结 

经过以上实验，我有如下两个较大的疑问：<br>
* 为什么多特征训练后的结果没有单一特征v训练出的结果好？我仔细检查过代码了，应该不是代码错误导致的误差
* 为什么模型融合后的结果没有融合前的结果好？
<br>

这是我的猜想：<br>

* 各个特征的重要程度是不同的，而将不太重要的特征如stops_time和很重要的特征v一同训练，反倒形成了干扰。比如通过查看源码，可知仅在聚类操作这一环节就会形成很大的干扰。同时特征多的话，特征之间的组合就更多，也就理应需要设置更多的隐含状态个数。但是由于计算量方面的考虑，我并没有增加隐含状态的个数，导致隐含状态的分类过于粗放，产生了巨大的误差。
* 单个车辆的历史记录太少，如上次实验(见hmm文件夹)推断，这应该是一天中一段时间的轨迹数据。这样的数据规律性不强，反倒对原模型造成干扰。如果有数天的该车辆的行车数据，可能就能习得该车辆的行车习惯，这时进行模型融合可能会有更好的效果

解决方案：<br>

* 对各个特征分别建立隐马尔可夫模型，预测的时候取各个模型预测的参数，再使用这些参数算得预测结果。或想办法将数据加权之后再建立隐马尔可夫模型，但是这涉及调参的问题
* 对该问题而言，只能放弃这种方法

## 下一步工作

自己写出hmm的各种算法，替换使用的hmmlearn库<br>
我一直在找相关的代码，但教程、博客上的代码往往很简单或语焉不详，<br>
不过即便这样，也应该是可以一步一步写的。<br>
但主要问题是，这里的观测量是连续的，不是离散的，而关于建立观测量为连续量的隐马尔可夫模型的文章几乎没有，更不用说有代码示例的相关文章了<br>
包括李航博士的《统计学习方法》上也没有讲如何建立观测量为连续量的隐马尔可夫模型<br>
所以万般无奈下，我这几天在研究hmmlearn库的源码，但这对我有一定的难度，估计还要不少时间才能自己写出这些代码

## 其他

我用了两天用训练集的v特征训练出了一个迭代500次的模型，按理说效果应该是所有模型中最优的。但在我打开页面的时候，浏览器加载不下来，电脑几近卡死，导致我的Python核心崩溃，最终没有得到这个模型。<br>
现在我还有一个多特征的较高精度模型正在训练，但我估计这个模型的效果不会很好<br>
<br>
这是我第一次处理这么大的数据，也是第一次使用部署在服务器上的jupyter，出现了很多意料之外的问题，花掉了许多时间，但熟悉之后，就没什么问题了。